In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import pandas_gbq
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
project_id = "perceptive-ivy-290216"

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
query2=f"""
SELECT * FROM `perceptive-ivy-290216.f1_api.race_lap_time`
# WHERE YEAR=2024
# AND GP='Saudi Arabian Grand Prix'
"""
track3=pandas_gbq.read_gbq(query2,project_id,dialect='standard')

Downloading: 100%|██████████|


In [4]:
query_all_laps=f"""
SELECT * FROM `perceptive-ivy-290216.f1_api.race_lap_time`
UNION ALL
SELECT * FROM `perceptive-ivy-290216.f1_api.final_driver_positions_all`
"""
track_all_laps=pandas_gbq.read_gbq(query_all_laps,project_id,dialect='standard')

Downloading: 100%|██████████|


In [5]:
race="Belgian Grand Prix"

In [143]:
track2=track3[(track3["GP"]==race)&(track3["Year"]==2024)]

In [144]:
#Get Final Pos to order the final plot
track_all_laps_2=track_all_laps[(track_all_laps["GP"]==race)&(track_all_laps["Year"]==2024)]
final_pos=track_all_laps_2[track_all_laps_2["LapNumber"]==track_all_laps_2.LapNumber.max()][["Driver","Position"]]

In [146]:
year=track2['Year'].iloc[0]
gp=track2['GP'].iloc[0]
Drivers=(track2['Driver'].unique())

In [147]:
stints = track2[["Driver", "Stint", "Compound", "LapNumber"]]
stints = stints.groupby(["Driver", "Stint", "Compound"]).count().reset_index()
stints = stints.rename(columns={"LapNumber": "StintLength"})
stints.head()

,Driver,Stint,Compound,StintLength
0,ALB,1.0,MEDIUM,8
1,ALB,2.0,MEDIUM,15
2,ALB,3.0,HARD,21
3,ALO,1.0,MEDIUM,13
4,ALO,2.0,HARD,31


In [148]:
stint_with_pos=stints.merge(final_pos, left_on=["Driver"],right_on=["Driver"]).sort_values(by=['Position','Stint'])
stint_with_pos.head()

,Driver,Stint,Compound,StintLength,Position
38,RUS,1.0,MEDIUM,10,1.0
39,RUS,2.0,HARD,34,1.0
11,HAM,1.0,MEDIUM,11,2.0
12,HAM,2.0,HARD,15,2.0
13,HAM,3.0,HARD,18,2.0


In [149]:
fig=px.bar(
    stint_with_pos,
    y="Driver",
    x='StintLength',
    text="StintLength",
    orientation='h',
    barmode='stack',
    color='Compound',
    template="simple_white",
    title="<b>Tyre Strategies for the {} {}</b>".format(year,gp),
    height=800, width=1200,
    hover_data=["Position"],
    color_discrete_map={
                 "SOFT": "#F91536",
                 "MEDIUM": "#ffdf00 ",
                 "HARD":"#c8c8c8 ",
                 "INTERMEDIATE":"#009E60 ",
                 "WET":"#3671C6"
    }
)
fig.update_layout(
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    xaxis_title="<b>Lap Number</b>",
    yaxis_title="<b>Driver</b>",
    title_x=0.5,
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)

fig.update_layout(yaxis={'categoryorder':'total descending'},    
        hoverlabel=dict(
        bgcolor="Black",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    margin=dict(l=5, r=5, t=35, b=5),
    )

fig['layout']['yaxis']['autorange'] = "reversed"

fig.update_traces(marker_line_width=1, marker_line_color="WHITE", opacity=0.99, insidetextanchor="middle",textfont_color="White")

fig.update_yaxes(ticksuffix = "  ")

fig.show()

In [150]:
fig.write_html("{} {} Tyre Strategies.html".format(year,gp))

In [ ]:
#Multi Tracks

In [8]:
#2023
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Las Vegas Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Pre-Season Testing","Qatar Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2022
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2021
# race_multi=["Abu Dhabi Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Mexico City Grand Prix","Monaco Grand Prix","Portuguese Grand Prix","Qatar Grand Prix","Russian Grand Prix","Saudi Arabian Grand Prix","Spanish Grand Prix","Styrian Grand Prix","São Paulo Grand Prix","Turkish Grand Prix","United States Grand Prix"]
#2020
race_multi=["70th Anniversary Grand Prix","Abu Dhabi Grand Prix","Austrian Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Eifel Grand Prix","Emilia Romagna Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Portuguese Grand Prix","Russian Grand Prix","Sakhir Grand Prix","Spanish Grand Prix","Styrian Grand Prix","Turkish Grand Prix","Tuscan Grand Prix"]

year_multi=2020
for i in race_multi:
    track2=track3[(track3["GP"]==i)&(track3["Year"]==year_multi)]
    track_all_laps_2=track_all_laps[(track_all_laps["GP"]==i)&(track_all_laps["Year"]==year_multi)]
    final_pos=track_all_laps_2[track_all_laps_2["LapNumber"]==track_all_laps_2.LapNumber.max()][["Driver","Position"]]
    year=track2['Year'].iloc[0]
    gp=track2['GP'].iloc[0]
    Drivers=(track2['Driver'].unique())
    stints = track2[["Driver", "Stint", "Compound", "LapNumber"]]
    stints = stints.groupby(["Driver", "Stint", "Compound"]).count().reset_index()
    stints = stints.rename(columns={"LapNumber": "StintLength"})
    stints.head()
    stint_with_pos=stints.merge(final_pos, left_on=["Driver"],right_on=["Driver"]).sort_values(by=['Position','Stint'])
    stint_with_pos.head()
    fig=px.bar(
        stint_with_pos,
        y="Driver",
        x='StintLength',
        text="StintLength",
        orientation='h',
        barmode='stack',
        color='Compound',
        template="simple_white",
        title="<b>Tyre Strategies for the {} {}</b>".format(year_multi,i),
        height=800, width=1200,
        hover_data=["Position"],
        color_discrete_map={
                    "SOFT": "#F91536",
                    "MEDIUM": "#ffdf00 ",
                    "HARD":"#c8c8c8 ",
                    "INTERMEDIATE":"#009E60 ",
                    "WET":"#3671C6"
        }
    )
    fig.update_layout(
        yaxis = dict(tickfont = dict(size=20)),
        xaxis = dict(tickfont = dict(size=20)),
        xaxis_title="<b>Lap Number</b>",
        yaxis_title="<b>Driver</b>",
        title_x=0.5,
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )

    fig.update_layout(yaxis={'categoryorder':'total descending'},    
            hoverlabel=dict(
            bgcolor="Black",
            font_size=16,
            font_family="PT Sans Narrow"
        ),
        margin=dict(l=5, r=5, t=35, b=5),
        )

    fig['layout']['yaxis']['autorange'] = "reversed"

    fig.update_traces(marker_line_width=1, marker_line_color="WHITE", opacity=0.99, insidetextanchor="middle",textfont_color="White")

    fig.update_yaxes(ticksuffix = "  ")
    fig.write_html("{} {} Tyre Strategies.html".format(year_multi,i))
